In [2]:
import pickle
import json

import re

In [3]:
def concat_conversation(data, labels, sep = ' | ', label_type = '_sent_c99_label'):
    conversations = []
    summaries = []
    for i in range(0, len(data)):
        if len(data[i]['dialogue'].split('\r\n')) > 1:
            sentences = data[i]['dialogue'].replace(" |", " ").split('\r\n')
            
        else:
            sentences = data[i]['dialogue'].replace(" |", " ").split('\n')
            
        if len(sentences) == 1:
            continue
         
        if 'summary' in data[i]:
            summaries.append(data[i]['summary'].strip('\n').replace('\r\nt', ' '))
        else:
            summaries.append(data[i]['summary1'].strip('\n').replace('\r\nt', ' '))

        if len(labels) > 1:
            
            if label_type == '_sent_c99_label':
                temp = ''
                temp += sentences[0]
                for j in range(1, len(sentences)):
                    if labels[i][j] != labels[i][j-1]:

                        temp = temp + sep + sentences[j]
                    else:
                        temp = temp + ' ' + sentences[j]
                temp += ' | '
                conversations.append(temp)
            else:
                temp = ' | '
                temp += sentences[0]
                for j in range(1, len(sentences)):
                    if labels[i][j] != labels[i][j-1]:

                        temp = temp + sep + sentences[j]
                    else:
                        temp = temp + ' ' + sentences[j]
                conversations.append(temp)
                
        elif labels[0] == 1:
            conversations.append(' | ' + ' | '.join(sentences))
        elif labels[0] == 0:
            conversations.append(' | ' + ' '.join(sentences))
        
    return conversations, summaries
    
def read_json(file):
    json_arr = []
    lines = file.readlines()
    for line in lines:
        jsn = json.loads(line)
        json_arr.append(jsn)
    return json_arr
    
    
def transform_format(prefix, label_type = '_sent_c99_label'):
    with open(prefix + '.jsonl', encoding = 'utf8') as json_file:
        data = read_json(json_file)
        #data = json.load(json_file)
    if label_type != '_all' and label_type != '_none':
        with open(prefix + label_type +'.pkl', 'rb') as f:
            labels = pickle.load(f)
    elif label_type == '_all':
        labels = [1]
    elif label_type == '_none':
        labels = [0]
        
    cons, sums = concat_conversation(data, labels, label_type)    

    with open(prefix + label_type +'.source', 'wt', encoding='utf-8') as source_file, open(prefix + label_type + '.target', 'wt', encoding='utf-8') as target_file:
        for i in range(0, len(cons)):
            article = cons[i]
            abstract = sums[i]
            if '\n' in abstract:
                abstract = ' '.join(abstract.split('\n'))
            source_file.write(article + '\n')
            target_file.write(abstract + '\n')
    

In [6]:
train_prefix = './dialogsum/DialogSum_Data/train_dialogsum'
dev_prefix = './dialogsum/DialogSum_Data/val_dialogsum'
test_prefix = './dialogsum/DialogSum_Data/test_dialogsum'

In [46]:
for u in [train_prefix, dev_prefix, test_prefix]:
    transform_format(u, '_all')
    transform_format(u, '_none')
    transform_format(u, '_sent_trans_cons_label_2') 
    transform_format(u, '_sent_c99_label') 

In [4]:
def transform_format_alt(prefix, label_type = '_sent_c99_label'):
    with open(prefix + '.jsonl', encoding = 'utf8') as json_file:
        data = read_json(json_file)
        #data = json.load(json_file)
    if label_type != '_all' and label_type != '_none':
        with open(prefix + label_type +'.pkl', 'rb') as f:
            labels = pickle.load(f)
    elif label_type == '_all':
        labels = [1]
    elif label_type == '_none':
        labels = [0]
        
    cons, sums = concat_conversation(data, labels, label_type)    
    
    return cons

In [ ]:
transform_format_alt(dev_prefix, '_all')